In [ ]:
import sys
!{sys.executable} -m pip install hdfs

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [19]:
# read data from the silver tables
players_df = spark.table("NBA.sl_players_stats")


In [ ]:
players_df.printSchema()

In [ ]:
players_df.toPandas()

In [ ]:
players_df.createOrReplaceTempView("players_df")

# Then you can write all the SQL you want, as your json is seen as a regular table now.
sqlized_df = spark.sql(
    """
    SELECT player, name_never_player,name ,pick_overall,Team, team_id
    FROM players_df
    WHERE Team is not null or team_id is not null and draft_year is not null
    """
)
sqlized_df.toPandas()

In [ ]:
players_df.createOrReplaceTempView("players_df")

# Then you can write all the SQL you want, as your json is seen as a regular table now.
sqlized_df = spark.sql(
    """
    SELECT *
    FROM players_df
    WHERE (team_id is not null) and (player is not null or name is not null) 
    """
)
sqlized_df.toPandas()

In [ ]:
#anos de adaptação dos jogadores 

In [ ]:
df_idade4.createOrReplaceTempView("df_idade4")

# Then you can write all the SQL you want, as your json is seen as a regular table now.
sqlized_df = spark.sql(
    """
    SELECT player,seasons
    FROM df_idade4
    WHERE 
    """
)
sqlized_df.toPandas()
#existe 3558 linhas para tratar dos anos de adaptção dos jogadores

In [ ]:
#é necessario agregar por nome

In [20]:
#drop de tudo o que tem none
df_idade= players_df.drop('Team')
df_idade= df_idade.drop('College')
df_idade= df_idade.drop('games_NA')
df_idade= df_idade.drop('year_never_player')
df_idade= df_idade.drop('pick_never_played')

In [21]:
#agora é necessario filtrar para usar apenas os jogadores do sql anterior 
from pyspark.sql.functions import when, col, concat, lit
df_idade2 =df_idade.where(col("age_draft").isNotNull())


In [ ]:
df_idade2.toPandas()

In [ ]:
#agora é necessario agregar os jogadores por linha 

In [22]:
#primeiramente preciso de adicionar uma coluna contador que conta o numero de epocas do jogador
from pyspark.sql.functions import when, col, concat, lit

df_idade3 = df_idade2.withColumn(    
    "contador_epocas",
    lit(1)
)


In [ ]:
df_idade3.toPandas()

In [23]:
from pyspark.sql.functions import substring, avg, sum

df_idade4 = df_idade3 \
    .groupBy("player", "pick_overall", "draft_year","pos","college_name", "games_total_seasons", "ano", "age_draft") \
    .agg(
        sum(df_idade3.contador_epocas).cast('INT').alias("contador_epocas"))

In [ ]:
df_idade4.toPandas()

In [ ]:
df_idade4.printSchema()

In [ ]:
#criação da primeira tabela 
spark.sql(
    """
    CREATE DATABASE IF NOT EXISTS NBA LOCATION 'hdfs://hdfs-nn:9000/projeto/NBA.db'
    """
)
spark.sql(
    """
    DROP TABLE IF EXISTS NBA.overall_stats_gold
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE NBA.overall_stats_gold (
        player string ,
        pick_overall integer ,
        draft_year integer ,
        pos string ,
        college_name string , 
        games_total_seasons string ,
        ano integer ,
        age_draft DOUBLE,
        contador_epocas integer
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/projeto/NBA.db/overall_stats_gold/'
    """
)

In [ ]:
df_idade4 \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/projeto/NBA.db/overall_stats_gold/")

In [ ]:
spark.table("NBA.overall_stats_gold").show()

In [ ]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/projeto/NBA.db/overall_stats_gold/`
""").show()

In [ ]:
spark.sql("""
DROP TABLE IF EXISTS NBA.overall_stats_gold_presto
""").show()

spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS NBA.overall_stats_gold_presto(
  player string ,
        pick_overall integer ,
        draft_year integer ,
        pos string ,
        college_name string , 
        games_total_seasons string ,
        ano integer ,
        age_draft DOUBLE,
        contador_epocas integer
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/projeto/NBA.db/overall_stats_gold/_symlink_format_manifest/'
""").show()

In [ ]:
df_idade4.createOrReplaceTempView("df_idade4")

# Then you can write all the SQL you want, as your json is seen as a regular table now.
sqlized_df = spark.sql(
    """
    SELECT player
    FROM df_idade4
    WHERE pick_overall== 1
    """
)
sqlized_df.toPandas()

In [ ]:
#necessario fazer a agregação por picks, sendo que ha
players_df.toPandas()

# COMPARAÇÃO DE JOGADORES

o reference so das as 14 primeiras picks. Se houver associação com o b_reference significa que foi a primeira 14
e jogou
se tiver so b_reference significa que foi fora das 14 e jogou

posso tratar primeiro os dados dos primeiros 14 
depois outra tabela que compare os 14 com os que nao jogaram 
Percentagem dos que jogaram e assim 

In [24]:
#drop de tudo o que tem none
df_best= players_df.drop('Team')
df_best= df_best.drop('College')
df_best= df_best.drop('games_NA')
df_best= df_best.drop('year_never_player')
df_best= df_best.drop('pick_never_played')

In [ ]:
df_best.printSchema()

In [25]:
from pyspark.sql.functions import substring, avg, sum

df_best1 = df_best.filter((df_best.reference != "null") & (df_best.b_reference != "null")) 

In [ ]:
df_best1.toPandas()

In [26]:
from pyspark.sql.functions import substring, avg, sum

df_best2= df_best1 \
.groupBy("player","pos","pick_overall")\
.agg(
    sum(df_best1.games_total_seasons).cast('INT').alias("games_total_seasons"),
    avg(df_best1.percentage_minutes_game).alias("percentage_minutes_game_avg"),
    avg(df_best1.percentage_minutes_game_season).alias("percentage_minutes_game_season_avg"),
    avg(df_best1.vorp).alias("vorp_avg"),
    avg(df_best1.bpm).alias("bpm_avg"),
    avg(df_best1.player_efficiency).alias("player_efficiency"),
    avg(df_best1.win_shares).alias("win_shares")
)



In [ ]:
df_best1.printSchema()

In [ ]:
df_best2.toPandas()

In [27]:
    import pyspark.sql.functions as func
    df_best3 = df_best2 \
    .withColumn("percentage_minutes_game_avg", func.round(df_best2["percentage_minutes_game_avg"], 3)) \
    .withColumn("percentage_minutes_game_season_avg", func.round(df_best2["percentage_minutes_game_season_avg"], 3)) \
    .withColumn("vorp_avg", func.round(df_best2["vorp_avg"], 3)) \
    .withColumn("bpm_avg", func.round(df_best2["bpm_avg"], 3)) \
    .withColumn("player_efficiency", func.round(df_best2["player_efficiency"], 3)) \
    .withColumn("win_shares", func.round(df_best2["win_shares"], 3)) 
    

In [ ]:
df_best3.printSchema()

In [ ]:
#criação da primeira tabela 
spark.sql(
    """
    CREATE DATABASE IF NOT EXISTS NBA LOCATION 'hdfs://hdfs-nn:9000/projeto/NBA.db'
    """
)
spark.sql(
    """
    DROP TABLE IF EXISTS NBA.first_picks_gold
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE NBA.first_picks_gold(
        player string, 
        pos string,
        pick_overall integer  ,
        games_total_seasons integer ,
        percentage_minutes_game_avg double ,
        percentage_minutes_game_season_avg double ,
        vorp_avg double , 
        bpm_avg double,
        player_efficiency double, 
        win_shares double
        
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/projeto/NBA.db/first_picks_gold/'
    """
)

In [ ]:
df_best3 \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/projeto/NBA.db/first_picks_gold/")

In [ ]:
spark.table("NBA.first_picks_gold").show()

In [ ]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/projeto/NBA.db/first_picks_gold/`
""").show()

In [ ]:
spark.sql("""
DROP TABLE IF EXISTS NBA.first_picks_gold_presto
""").show()

spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS NBA.first_picks_gold_presto(

       player string, 
        pos string,
        pick_overall integer  ,
        games_total_seasons integer ,
        percentage_minutes_game_avg double ,
        percentage_minutes_game_season_avg double ,
        vorp_avg double , 
        bpm_avg double,
        player_efficiency double, 
        win_shares double
        
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/projeto/NBA.db/first_picks_gold/_symlink_format_manifest/'
""").show()

# Jogadores todos

In [ ]:
players_df.toPandas()

In [28]:
players_df.createOrReplaceTempView("players_df")

# Then you can write all the SQL you want, as your json is seen as a regular table now.
df_all = spark.sql(
    """
    SELECT *
    FROM players_df
    WHERE b_reference is  null and reference is not null
    """
)


In [ ]:
df_all.createOrReplaceTempView("df_all")

# Then you can write all the SQL you want, as your json is seen as a regular table now.
sqlized_df = spark.sql(
    """
    SELECT *
    FROM df_all
    where pick_overall is not null
    """
)

sqlized_df.toPandas()

In [ ]:
df_all.toPandas()

In [29]:
df_all1= df_all.drop('age_draft')
df_all1= df_all1.drop('ano')
df_all1= df_all1.drop('id')
df_all1= df_all1.drop('pick_never_played')
df_all1= df_all1.drop('year_never_player')
df_all1= df_all1.drop('name_never_player')
df_all1= df_all1.drop('games_total_seasons')
df_all1= df_all1.drop('Team')
df_all1= df_all1.drop('pick_overall')
df_all1= df_all1.drop('b_reference')
df_all1= df_all1.drop('draft_year')
df_all1= df_all1.drop('name')
df_all1= df_all1.drop('college_name')
df_all1= df_all1.drop('seasons')
df_all1= df_all1.drop('College')
df_all1= df_all1.drop('games_NA')



In [ ]:
df_all1.toPandas()

In [ ]:
df_all1.printSchema()

In [30]:
from pyspark.sql.functions import substring, avg, sum

df_all2 = df_all1 \
    .groupBy("player", "pos") \
    .agg(
        sum(df_all1.games).cast('INT').alias("games"),
        avg(df_all1.player_efficiency).alias("player_efficiency"),
        avg(df_all1.percentage_minutes_game_season).alias("percentage_minutes_game_season_avg"),
        avg(df_all1.vorp).alias("vorp_avg"),
        avg(df_all1.bpm).alias("bpm_avg"),
        avg(df_all1.minutes_played).alias("minutes_played"),
        avg(df_all1.win_shares).alias("win_shares")
)

In [ ]:
df_all2.printSchema()

In [ ]:
#criação da primeira tabela 
spark.sql(
    """
    CREATE DATABASE IF NOT EXISTS NBA LOCATION 'hdfs://hdfs-nn:9000/projeto/NBA.db'
    """
)
spark.sql(
    """
    DROP TABLE IF EXISTS NBA.seconds_picks_gold
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE NBA.seconds_picks_gold (
        player string ,
        pos string ,
        games integer ,
        player_efficiency double ,
        percentage_minutes_game_season_avg double  , 
        vorp_avg double ,
        bpm_avg double ,
        minutes_played double,
        win_shares double
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/projeto/NBA.db/seconds_picks_gold/'
    """
)

In [ ]:
df_all2 \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/projeto/NBA.db/seconds_picks_gold/")

In [ ]:
spark.table("NBA.seconds_picks_gold").show()

In [ ]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/projeto/NBA.db/seconds_picks_gold/`
""").show()

In [ ]:
spark.sql("""
DROP TABLE IF EXISTS NBA.seconds_picks_gold_presto
""").show()

spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS NBA.seconds_picks_gold_presto(
        player string ,
        pos string ,
        games integer ,
        player_efficiency double ,
        percentage_minutes_game_season_avg double  , 
        vorp_avg double ,
        bpm_avg double ,
        minutes_played double,
        win_shares double
        
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/projeto/NBA.db/seconds_picks_gold/_symlink_format_manifest/'
""").show()

TABELA DE COMPARAÇÃO DOS 14 QUE JOGARAM E NÃO JOGARAM 

In [ ]:
players_df.toPandas()

In [31]:
players_df.createOrReplaceTempView("players_df")

# Then you can write all the SQL you want, as your json is seen as a regular table now.
df_comparacao = spark.sql(
    """
    SELECT b_reference, name
    FROM players_df
    where b_reference is not null and reference is  null
    """
)

df_comparacao.toPandas()
#numero de jogadores que foram as primeiras escolhas do draft mas nunca jogaram

b_reference               name
0    okorois01        Isaac Okoro
1    nesmiaa01      Aaron Nesmith
2    hayeski01      Killian Hayes
3    okongon01     Onyeka Okongwu
4    smithja04        Jalen Smith
5    vazqufr01       Fran Vázquez
6    edwaran01    Anthony Edwards
7    vassede01      Devin Vassell
8    avdijde01        Deni Avdija
9    wisemja01      James Wiseman
10    ballla01        LaMelo Ball
11   toppiob01         Obi Toppin
12   halibty01  Tyrese Haliburton
13   willipa01   Patrick Williams
14   lewiski01         Kira Lewis

In [ ]:
#criação da primeira tabela 
spark.sql(
    """
    CREATE DATABASE IF NOT EXISTS NBA LOCATION 'hdfs://hdfs-nn:9000/projeto/NBA.db'
    """
)
spark.sql(
    """
    DROP TABLE IF EXISTS NBA.comparation_gold
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE NBA.comparation_gold (
        name string,
        b_reference string
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/projeto/NBA.db/comparation_gold/'
    """
)

In [ ]:
df_comparacao \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/projeto/NBA.db/comparation_gold/")

In [ ]:
spark.table("NBA.comparation_gold").show()

In [ ]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/projeto/NBA.db/comparation_gold/`
""").show()

In [ ]:
spark.sql("""
DROP TABLE IF EXISTS NBA.comparation_gold_presto
""").show()

spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS NBA.comparation_gold_presto(
         name string,
        b_reference string
        
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/projeto/NBA.db/comparation_gold/_symlink_format_manifest/'
""").show()

EQUIPAS

In [32]:
players_df.createOrReplaceTempView("players_df")

# Then you can write all the SQL you want, as your json is seen as a regular table now.
equipas_df = spark.sql(
    """
    SELECT *
    FROM players_df
    WHERE (team_id is not null) and (player is not null or name is not null) 
    """
)


In [ ]:
equipas_df.printSchema()

In [33]:
from pyspark.sql.functions import substring, avg, sum

equipas_df1 = equipas_df \
    .groupBy("player", "pos", "name", "year", "team_id","draft_year") \
    .agg(
        sum(equipas_df.games).cast('INT').alias("games"),
        avg(equipas_df.true_shooting_percentage).alias("true_shooting_percentage"),
        avg(equipas_df.percentage_minutes_game_season).alias("percentage_minutes_game_season_avg"),
        avg(equipas_df.vorp).alias("vorp_avg"),
        avg(equipas_df.bpm).alias("bpm_avg"),
        avg(equipas_df.minutes_played).alias("minutes_played"),
        avg(equipas_df.win_shares).alias("win_shares"),
        avg(equipas_df.player_efficiency).alias("player_efficiency"),
        avg(equipas_df.pick_overall).alias("pick_overall")
    
)

In [ ]:
equipas_df1.toPandas()

In [ ]:
equipas_df1.printSchema()

In [ ]:
#criação da primeira tabela 
spark.sql(
    """
    CREATE DATABASE IF NOT EXISTS NBA LOCATION 'hdfs://hdfs-nn:9000/projeto/NBA.db'
    """
)
spark.sql(
    """
    DROP TABLE IF EXISTS NBA.teams_picks_gold
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE NBA.teams_picks_gold (
        player string ,
        pos string ,
        name string,
        year int,
        team_id string, 
        draft_year int, 
        games int,
        true_shooting_percentage double,
        percentage_minutes_game_season_avg double,
        vorp_avg double,
        bpm_avg double,
        minutes_played double,
        win_shares double, 
        player_efficiency double, 
        pick_overall double
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/projeto/NBA.db/teams_picks_gold/'
    """
)

In [ ]:
equipas_df1 \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/projeto/NBA.db/teams_picks_gold/")

In [ ]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/projeto/NBA.db/teams_picks_gold/`
""").show()

In [ ]:
spark.sql("""
DROP TABLE IF EXISTS NBA.teams_picks_gold_presto
""").show()

spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS NBA.teams_picks_gold_presto(
                player string ,
        pos string ,
        name string,
        year int,
        team_id string, 
        draft_year int, 
        games int,
        true_shooting_percentage double,
        percentage_minutes_game_season_avg double,
        vorp_avg double,
        bpm_avg double,
        minutes_played double,
        win_shares double, 
        player_efficiency double, 
        pick_overall double
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/projeto/NBA.db/teams_picks_gold/_symlink_format_manifest/'
""").show()